In [ ]:
# enabling 3rd party widgets
# from google.colab import output
# output.enable_custom_widget_manager()
# output.disable_custom_widget_manager()

# interactive 3D plot
# !pip install ipympl
# %matplotlib widget

In [ ]:
import os
import math
from collections import OrderedDict
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

import time as time
import platform as platform

import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import L2

In [ ]:
current_sys = platform.system()

if current_sys == 'Windows':
    dir_sep = '\\'
else:
    dir_sep = '/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Thesis/Lorenz/')
print(os.getcwd())

/content/drive/MyDrive/Colab Notebooks/Thesis/Lorenz


In [ ]:
# setting seed for PRNGs
prng_seed = 42
np.random.seed(prng_seed)
tf.random.set_seed(prng_seed)

In [ ]:
tf.test.gpu_device_name()

''

# Lorenz System

In [ ]:
from tools.misc_tools import create_Lorenz_data

In [ ]:
# setting up params
sigma_arr = np.array([10, 15, 20])
rho_arr = np.array([20, 25, 30])
beta_arr = np.array([4/3, 6/3, 8/3])

x0 = 1
y0 = 1
z0 = 1

t0 = 0.0
T = 100.0
delta_t = 0.01

return_params_arr = False
normalize_flag = True

In [ ]:
res_dict = create_Lorenz_data(
    T, t0, delta_t,
    rho_arr, sigma_arr, beta_arr,
    x0, y0, z0, return_params_arr=return_params_arr,
    normalize=normalize_flag
)

all_data = res_dict['all_data']
N = res_dict['N']
boundary_idx_arr = res_dict['boundary_idx_arr']

if return_params_arr == True:
    params_arr = res_dict['params_arr']

if normalize_flag == True:
    normalization_constant_arr = res_dict['normalization_constant_arr']

In [ ]:
n = len(boundary_idx_arr)
# # '''
# num_cols = 1
# num_rows = n

# fig = plt.figure(figsize=(7.5*num_cols, 7.5*num_rows))

# prev_idx = 0
# for i in range(n):
#     # ax = plt.axes(projection ='3d')
#     next_idx = boundary_idx_arr[i]
    
#     ax_orig = fig.add_subplot(num_rows, num_cols, i+1, projection ='3d')
#     ax_orig.plot(all_data[prev_idx:next_idx, 0], all_data[prev_idx:next_idx, 1], all_data[prev_idx:next_idx, 2])
#     ax_orig.title.set_text(r'Actual Data - [$\sigma$, $\rho$, $\beta$] = ' + str(all_data[next_idx-1, 3:]))
#     ax_orig.set_xlabel('x')
#     ax_orig.set_ylabel('y')
#     ax_orig.set_zlabel('z')
    
#     # ax_predict = fig.add_subplot(num_rows, num_cols, 2*i+2, projection ='3d')
#     # ax_predict.plot(reconstructed_data[prev_idx:next_idx, 0], reconstructed_data[prev_idx:next_idx, 1], reconstructed_data[prev_idx:next_idx, 2])
#     # ax_predict.title.set_text(r'NN Reconstructed Data - [$\sigma$, $\rho$, $\beta$] = ' + str(all_data[next_idx-1, 3:])
#     # )
#     # ax_predict.set_xlabel('x')
#     # ax_predict.set_ylabel('y')
#     # ax_predict.set_zlabel('z')

#     prev_idx = next_idx
# # '''

# Autoencoder

In [ ]:
# setting up data
train_split = 0.8
val_split = 0.1  # to be used later on;
                 # `val_split` of total data will
                 # be taken out of `training_data`
                 # to use as validation data.
test_split = 1 - train_split - val_split


idx = np.arange(all_data.shape[0])
np.random.shuffle(idx)
boundary = int(np.round((1-test_split)*all_data.shape[0]))
training_data = all_data[idx[0:boundary], :]
testing_data = all_data[idx[boundary:], :]

# train_dataset = tf.data.Dataset.from_tensor_slices((training_data, training_data))
# test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_data))

batch_size = 64
# SHUFFLE_BUFFER_SIZE = 100

# # train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
# train_dataset = train_dataset.batch(BATCH_SIZE)
# test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
from tools.ae_v3 import Autoencoder

In [ ]:
# Training parameters
learning_rate_list = [0.001, 0.0001, 0.00001]
epochs = 2000
patience = 200  # parameter for early stopping
min_delta = 1e-6  # parameter for early stopping
lambda_reg = 1e-5  # weight for regularizer

# Initialize network
ae_net = Autoencoder(
    data_dim=6,
    enc_layers=[16,12,8,8,4,4,2],
    dec_layers=[2,4,4,8,8,12,16],
    latent_space_dim=2,
    lambda_reg=lambda_reg,
    reg_name='L2',
    enc_layer_act_func='elu',
    enc_final_layer_act_func='tanh',
    dec_layer_act_func='elu',
    dec_final_layer_act_func='linear',
    # batch_norm=True,
    load_file=None)

In [ ]:
# saving the autoencoder
dir_name_ae = os.getcwd() + '/saved_ae'
if not os.path.isdir(dir_name_ae):
    os.makedirs(dir_name_ae)

counter = 0
while True:
    dir_check = 'ae_' + str(counter).zfill(3)
    if os.path.isdir(dir_name_ae + '/' + dir_check):
        counter += 1
    else:
        break

dir_name_ae = dir_name_ae + '/' + dir_check
print('AE save dir : '+dir_name_ae)
os.makedirs(dir_name_ae)
os.makedirs(dir_name_ae+'/plots')

sim_data = {
    'rho_arr':rho_arr,
    'sigma_arr':sigma_arr,
    'beta_arr':beta_arr,
    'x0':x0,
    'y0':y0,
    'z0':z0,
    't0':t0,
    'T':T,
    'delta_t':delta_t,
    'return_params_arr':return_params_arr,
    'normalize_flag':normalize_flag
}

# import pandas as pd

# data_df = pd.DataFrame(data, columns=[key for key in sim_data.keys()])
# data_df.to_csv(dir_name_ae + '/sim_data_params.csv')

with open(dir_name_ae+'/sim_data_params.txt', 'w') as f:
    f.write(str(sim_data))


training_specific_params = {
    'learning_rate_list':learning_rate_list,
    'epochs':epochs,
    'patience':patience,
    'min_delta':min_delta,
    'prng_seed':prng_seed,
    'train_split':train_split,
    'val_split':val_split,
    'batch_size':batch_size
}

with open(dir_name_ae+'/training_specific_params.txt', 'w') as f:
    f.write(str(training_specific_params))

AE save dir : /content/drive/MyDrive/Colab Notebooks/Thesis/Lorenz/saved_ae/ae_001


In [ ]:
from tools.misc_tools import mytimecallback, SaveLosses

In [ ]:
# compiling the network
ae_net.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate_list[0]),
    loss=losses.MeanSquaredError()
)

# implementing early stopping
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=patience,
    restore_best_weights=True,
    verbose=True,
    min_delta=min_delta
)

# model checkpoint callback
dir_name_ckpt = dir_name_ae+'/checkpoints'
if not os.path.isdir(dir_name_ckpt):
    os.makedirs(dir_name_ckpt)
os.makedirs(dir_name_ckpt)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=dir_name_ckpt+dir_sep+'checkpoint',#+'/checkpoint--loss={loss:.4f}--vall_loss={val_loss:.4f}',
    monitor='val_loss',
    save_best_only=True,
    verbose=2,
    save_weights_only=True,
    period=5
)

timekeeper_cb = mytimecallback()

savelosses_cb_vallossarr = np.ones(shape=epochs*len(learning_rate_list))*np.NaN
savelosses_cb_trainlossarr = np.ones(shape=epochs*len(learning_rate_list))*np.NaN
savelosses_cb = SaveLosses(
    filepath=dir_name_ckpt+dir_sep+'LossHistoriesCheckpoint',
    val_loss_arr=savelosses_cb_vallossarr,
    train_loss_arr=savelosses_cb_trainlossarr,
    total_epochs=epochs,
    period=10)


# training the network
val_loss_hist = []
train_loss_hist = []
lr_change=[0]
for i in range(len(learning_rate_list)):
    learning_rate = learning_rate_list[i]
    K.set_value(ae_net.optimizer.lr, learning_rate)

    savelosses_cb.update_lr_idx(i)

    total_s_len = 80
    sep_lr_s = ' LEARNING RATE : {} '.format(learning_rate)
    sep_lr_s = int((total_s_len - len(sep_lr_s))//2)*'-' + sep_lr_s
    sep_lr_s = sep_lr_s + (total_s_len-len(sep_lr_s))*'-'
    print('\n\n' + '-'*len(sep_lr_s))
    print('\n' + sep_lr_s+'\n')
    print('-'*len(sep_lr_s) + '\n\n')
    
    history = ae_net.fit(training_data, training_data,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=val_split/train_split,
        callbacks=[early_stopping_cb, timekeeper_cb, checkpoint_cb, savelosses_cb],
        verbose=1
    )

    val_loss_hist.extend(history.history['val_loss'])
    train_loss_hist.extend(history.history['loss'])
    
    lr_change.append(lr_change[i]+len(history.history['val_loss']))

Streaming output truncated to the last 5000 lines.
3316/3323 [============================>.] - ETA: 0s - loss: 0.0032 - tot_time: 1h 53m 14.2s

Epoch 725: val_loss did not improve from 0.00299
3323/3323 [==============================] - 10s 3ms/step - loss: 0.0032 - val_loss: 0.0040
Epoch 726/2000
3323/3323 [==============================] - 12s 4ms/step - loss: 0.0032 - val_loss: 0.0050
Epoch 727/2000
3323/3323 [==============================] - 11s 3ms/step - loss: 0.0032 - val_loss: 0.0030
Epoch 728/2000
3323/3323 [==============================] - 11s 3ms/step - loss: 0.0032 - val_loss: 0.0033
Epoch 729/2000
3323/3323 [==============================] - 10s 3ms/step - loss: 0.0032 - val_loss: 0.0029
Epoch 730/2000
3317/3323 [============================>.] - ETA: 0s - loss: 0.0032 - tot_time: 1h 54m 9.0s

Epoch 730: val_loss did not improve from 0.00299
 - saving loss histories at /content/drive/MyDrive/Colab Notebooks/Thesis/Lorenz/saved_ae/ae_001/checkpoints/LossHistoriesCheckpo

In [ ]:
# plotting losses
from tools.misc_tools import plot_losses


test_loss = ae_net.evaluate(
    testing_data, testing_data,
)
# lr_change = [0, 987, 987+334]
# for i in range(1,len(lr_change)):
#     lr_change[i] += lr_change[i-1]

# Visualize loss history
fig, ax = plot_losses(
    training_loss=train_loss_hist,
    val_loss=val_loss_hist,
    lr_change=lr_change,
    learning_rate_list=learning_rate_list
)

plt.savefig(dir_name_ae+'/plots/loss_history.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
reconstructed_data = ae_net.predict(all_data)

In [ ]:
fig = plot_reconstructed_data(
    boundary_idx_arr=boundary_idx_arr,
    dir_name_ae=dir_name_ae,
    all_data=all_data,
    reconstructed_data=reconstructed_data,
    save_figs=False)

In [ ]:
plot_reconstructed_data(
    boundary_idx_arr=boundary_idx_arr,
    dir_name_ae=dir_name_ae,
    all_data=all_data,
    reconstructed_data=reconstructed_data,
    save_figs=True)

In [ ]:
# create data
latent_states_all = ae_net.encoder_net.predict(all_data)

In [ ]:
from tools.misc_tools import plot_latent_states

In [ ]:
fig, ax = plot_latent_states(
    boundary_idx_arr=boundary_idx_arr,
    latent_states_all=latent_states_all,
    all_data=all_data,
    xlim=[-1,1],
    ylim=[-1,1],
    cmap_name='gist_rainbow',
    legend_markerscale=10
    )

plt.savefig(dir_name_ae + '{ds}plots{ds}latent_space.png'.format(ds=dir_sep), dpi=300, bbox_inches='tight')

In [ ]:
# ae_net.summary()

In [ ]:
save_path = dir_name_ae+dir_sep+'final_net'

if not os.path.isdir(save_path):
    os.makedirs(save_path)

with open(save_path+dir_sep+'losses.txt', 'w') as f:
    f.write(str({
        'val_loss_hist':val_loss_hist,
        'train_loss_hist':train_loss_hist,
        'lr_change':lr_change,
        'test_loss':test_loss
    }))

ae_net.save_everything(file_name=save_path+dir_sep+'final_net')

In [ ]:
n = len(boundary_idx_arr)
num_cols = 1
num_rows = 3*n

# plt.ion()

fig = plt.figure(figsize=(7.5*num_cols, 7.5*num_rows))

Tt = N * delta_t
time_arr = np.arange(0, N+1) * Tt / N

y_labels = [r'$x_1$', r'$x_2$', r'$x_3$']

prev_idx = 0
for i in range(n):
    # ax = plt.axes(projection ='3d')
    next_idx = boundary_idx_arr[i]

    for j in range(3):
        ax = fig.add_subplot(num_rows, num_cols, 3*i+j+1)
        ax.plot(time_arr, all_data[prev_idx:next_idx, j], label='original')
        ax.plot(time_arr, reconstructed_data[prev_idx:next_idx, j], label='reconstructed')
        ax.set_ylabel(y_labels[j])
        ax.set_xlabel('time')
        ax.grid(True)


    prev_idx = next_idx

# LSTM